In [8]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [9]:
import matplotlib.pyplot as plt
import numpy as np
import os
import PIL
import cv2
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.python.keras.layers import Dense, Flatten
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from tensorflow.keras.preprocessing import image

In [10]:
def identity_block(x, filter):
    # copy tensor to variable called x_skip
    x_skip = x
    # Layer 1
    x = tf.keras.layers.Conv2D(filter, (3,3), padding = 'same')(x)
    x = tf.keras.layers.BatchNormalization(axis=3)(x)
    x = tf.keras.layers.Activation('relu')(x)
    # Layer 2
    x = tf.keras.layers.Conv2D(filter, (3,3), padding = 'same')(x)
    x = tf.keras.layers.BatchNormalization(axis=3)(x)
    # Add Residue
    x = tf.keras.layers.Add()([x, x_skip])
    x = tf.keras.layers.Activation('relu')(x)
    return x

In [11]:
def convolutional_block(x, filter):
    # copy tensor to variable called x_skip
    x_skip = x
    # Layer 1
    x = tf.keras.layers.Conv2D(filter, (3,3), padding = 'same', strides = (2,2))(x)
    x = tf.keras.layers.BatchNormalization(axis=3)(x)
    x = tf.keras.layers.Activation('relu')(x)
    # Layer 2
    x = tf.keras.layers.Conv2D(filter, (3,3), padding = 'same')(x)
    x = tf.keras.layers.BatchNormalization(axis=3)(x)
    # Processing Residue with conv(1,1)
    x_skip = tf.keras.layers.Conv2D(filter, (1,1), strides = (2,2))(x_skip)
    # Add Residue
    x = tf.keras.layers.Add()([x, x_skip])
    x = tf.keras.layers.Activation('relu')(x)
    return x

In [12]:
def ResNet34(shape = (180, 180, 3), classes = 2):
    # Step 1 (Setup Input Layer)
    x_input = tf.keras.layers.Input(shape)
    x = tf.keras.layers.ZeroPadding2D((3, 3))(x_input)
    # Step 2 (Initial Conv layer along with maxPool)
    x = tf.keras.layers.Conv2D(64, kernel_size=7, strides=2, padding='same')(x)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.Activation('relu')(x)
    x = tf.keras.layers.MaxPool2D(pool_size=3, strides=2, padding='same')(x)
    # Define size of sub-blocks and initial filter size
    block_layers = [3, 4, 6, 3]
    filter_size = 64
    # Step 3 Add the Resnet Blocks
    for i in range(4):
        if i == 0:
            # For sub-block 1 Residual/Convolutional block not needed
            for j in range(block_layers[i]):
                x = identity_block(x, filter_size)
        else:
            # One Residual/Convolutional Block followed by Identity blocks
            # The filter size will go on increasing by a factor of 2
            filter_size = filter_size*2
            x = convolutional_block(x, filter_size)
            for j in range(block_layers[i] - 1):
                x = identity_block(x, filter_size)
    # Step 4 End Dense Network
    x = tf.keras.layers.AveragePooling2D((2,2), padding = 'same')(x)
    x = tf.keras.layers.Flatten()(x)
    x = tf.keras.layers.Dense(512, activation = 'relu')(x)
    x = tf.keras.layers.Dense(1, activation = 'sigmoid')(x)
    model = tf.keras.models.Model(inputs = x_input, outputs = x, name = "ResNet34")
    return model

In [13]:
model = ResNet34()

In [14]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [15]:
model.load_weights('/content/drive/MyDrive/rope_skipping/runs/2000-epochs/weights/best.h5')

# Image

In [ ]:
image_path = '/content/1.jpg'
img = image.load_img(image_path, target_size=(180, 180))
img_array = image.img_to_array(img)
img_array = np.expand_dims(img_array, axis=0)

In [ ]:
predictions = model.predict(img_array)

1/1 [==============================] - 1s 944ms/step


In [ ]:
print("Predicted Probabilities:", predictions)

Predicted Probabilities: [[0.4811526]]


In [ ]:
predicted_class = 1 if predictions[0][0] >= 0.5 else 0
class_label = 'Skipping' if predicted_class == 1 else 'Not Skipping'
print("Predicted Class:", class_label)

Predicted Class: Not Skipping


# Video

In [16]:
def prediction(frame):
    # Convert the frame to an image array
    img_array = image.array_to_img(frame)
    img_array = img_array.resize((180, 180))  # Resize to match the target size
    img_array = image.img_to_array(img_array)
    img_array = np.expand_dims(img_array, axis=0)

    # Perform prediction
    predictions = model.predict(img_array)
    print(predictions)
    predicted_class = 1 if predictions[0][0] >= 0.5 else 0
    class_label = 'Skipping' if predicted_class == 1 else 'Not Skipping'
    return class_label

In [17]:
# def preprocess_frame(frame, target_size=(180, 180)):
#     frame = cv2.resize(frame, target_size)
#     frame_array = frame.astype(np.float32) / 255.0  # Normalize pixel values to [0, 1]
#     frame_array = np.expand_dims(frame_array, axis=0)  # Add batch dimension
#     return frame_array

In [18]:
def write_text(frame, text):
    font = cv2.FONT_HERSHEY_SIMPLEX
    org = (50, 50)
    fontScale = 1
    color = (0, 0, 255)
    thickness = 2
    cv2.putText(frame, text, org, font, fontScale, color, thickness, cv2.LINE_AA)

In [19]:
video_path = '/content/y2mate.com - X Skipping  Easy To Learn In 3 Steps  Sarpatta Dancing rose and Kabilan style  Sharas_1080p.mp4'  # Replace 'path_to_video_file' with the actual path of your video file
cap = cv2.VideoCapture(video_path)

In [20]:
output_file = '/content/output_video_test.avi'
frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = int(cap.get(cv2.CAP_PROP_FPS))
fourcc = cv2.VideoWriter_fourcc(*'XVID')
out = cv2.VideoWriter(output_file, fourcc, fps, (frame_width, frame_height))

In [21]:
# Assuming 'cap' is already defined and initialized
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break
    # Perform prediction on each frame
    result = prediction(frame)
    print("Predicted class:", result)

    write_text(frame,result )

    out.write(frame)

cap.release()
#cv2.destroyAllWindows()

1/1 [==============================] - 1s 1s/step
[[0.33488637]]
Predicted class: Not Skipping
1/1 [==============================] - 0s 127ms/step
[[0.3350633]]
Predicted class: Not Skipping
1/1 [==============================] - 0s 144ms/step
[[0.51512074]]
Predicted class: Skipping
1/1 [==============================] - 0s 128ms/step
[[0.5095683]]
Predicted class: Skipping
1/1 [==============================] - 0s 126ms/step
[[0.5696158]]
Predicted class: Skipping
1/1 [==============================] - 0s 132ms/step
[[0.25868857]]
Predicted class: Not Skipping
1/1 [==============================] - 0s 135ms/step
[[0.25170082]]
Predicted class: Not Skipping
1/1 [==============================] - 0s 139ms/step
[[0.02782157]]
Predicted class: Not Skipping
1/1 [==============================] - 0s 129ms/step
[[0.02179691]]
Predicted class: Not Skipping
1/1 [==============================] - 0s 130ms/step
[[0.00879995]]
Predicted class: Not Skipping
1/1 [==============================] -

In [ ]:
# while cap.isOpened():
#     ret, frame = cap.read()
#     if not ret:
#         break

#     preprocessed_frame = preprocess_frame(frame)

#     preprocessed_frame = np.squeeze(preprocessed_frame, axis=0)

#     print("Preprocessed Frame Shape:", preprocessed_frame.shape)

#     # Predict using the model
#     predictions = model.predict(np.expand_dims(preprocessed_frame, axis=0))
#     print(predictions)
#     predicted_class = 1 if predictions[0][0] >= 0.5 else 0
#     class_label = 'Skipping' if predicted_class == 1 else 'Not Skipping'
#     print("Predicted Class:", class_label)

#     write_text(frame, f'Predicted Class: {class_label}')

#     out.write(frame)

# cap.release()
# #cv2.destroyAllWindows()